In [ ]:
### 엣지별 버스정류장 혼잡도 데이터

import pandas as pd
import folium

In [ ]:
## 원하는 범위 내에 있는 버스 정류장의 위경도 데이터 수집


# 파일 경로 설정
total_by_stop_path = "C:/Users/pjy03/OneDrive/바탕 화면/university/KHUDA/심화프로젝트/혼잡정류장/total_by_stop.csv"  # total_by_stop 파일 경로
bus_stops_path = "C:/Users/pjy03/OneDrive/바탕 화면/university/KHUDA/심화프로젝트/혼잡정류장/국토교통부_전국 버스정류장 위치정보_20241028.csv"      # 버스정류소현황 파일 경로

# 데이터 불러오기
total_by_stop = pd.read_csv(total_by_stop_path, encoding='cp949')
bus_stops = pd.read_csv(bus_stops_path, encoding='cp949')

# 필요한 열만 선택 (정류소명, 위도, 경도)
bus_stops = bus_stops[['정류장명', '위도', '경도']]

# total_by_stop 데이터와 bus_stops 데이터를 정류소명(정류소 열 기준)으로 병합
merged_data = pd.merge(total_by_stop, bus_stops, left_on='정류장', right_on='정류장명', how='left')

# 정류소명 중복 열 제거
merged_data.drop(columns=['정류장명'], inplace=True)

# 병합 결과를 새로운 CSV 파일로 저장
merged_data.to_csv("total_by_stop_with_coordinates.csv", index=False, encoding='cp949')

print("정류소 데이터 병합 완료! 결과는 'total_by_stop_with_coordinates.csv'로 저장됨.")


In [ ]:
##각 엣지에 근접해 있는 정류장의 승하차 인원 합산

# Load the busstop edge file and bus stop data
busstop_edge_file = 'C:/Users/pjy03/OneDrive/바탕 화면/university/KHUDA/심화프로젝트/혼잡정류장/data_busstop_retry.csv'
busstop_data_file = 'C:/Users/pjy03/OneDrive/바탕 화면/university/KHUDA/심화프로젝트/혼잡정류장/total_by_stop_with_coordinates.csv'

busstop_edge_df = pd.read_csv(busstop_edge_file, encoding="cp949")
busstop_data_df = pd.read_csv(busstop_data_file, encoding='cp949')

# Ensure the required columns are present
edge_columns = ['id_start_lat', 'id_start_long', 'id_end_lat', 'id_end_long']
busstop_columns = ['latitude', 'longitude', 'total']

if not all(col in busstop_edge_df.columns for col in edge_columns):
    raise ValueError("Busstop edge file is missing required columns: " + str(edge_columns))

if not all(col in busstop_data_df.columns for col in busstop_columns):
    raise ValueError("Busstop data file is missing required columns: " + str(busstop_columns))

# Define a buffer for the rectangular boundary
buffer = 0.000005  # Approx 100 meters buffer, adjust as needed

# Initialize a new column for the total sum of 'total' values for each edge
busstop_edge_df['total_sum'] = 0

# Process each edge
for index, edge in busstop_edge_df.iterrows():
    # Get the edge boundaries with buffer
    min_lat = min(edge['id_start_lat'], edge['id_end_lat']) - buffer
    max_lat = max(edge['id_start_lat'], edge['id_end_lat']) + buffer
    min_lon = min(edge['id_start_long'], edge['id_end_long']) - buffer
    max_lon = max(edge['id_start_long'], edge['id_end_long']) + buffer

    # Filter bus stops within the boundary
    filtered_stops = busstop_data_df[
        (busstop_data_df['latitude'] >= min_lat) &
        (busstop_data_df['latitude'] <= max_lat) &
        (busstop_data_df['longitude'] >= min_lon) &
        (busstop_data_df['longitude'] <= max_lon)
    ]

    # Sum the 'total' values for the filtered stops
    busstop_edge_df.at[index, 'total_sum'] = filtered_stops['total'].sum()

# Save the updated busstop edge file
output_file = 'C:/Users/pjy03/OneDrive/바탕 화면/university/KHUDA/심화프로젝트/혼잡정류장/busstop_edited.csv'
busstop_edge_df.to_csv(output_file, index=False, encoding='utf-8')

# Create the map visualization
# Center map on the first edge's start location
start_location = [busstop_edge_df.iloc[0]['id_start_lat'], busstop_edge_df.iloc[0]['id_start_long']]
m = folium.Map(location=start_location, zoom_start=14)

# Add edges as lines
for _, edge in busstop_edge_df.iterrows():
    folium.PolyLine(
        locations=[
            [edge['id_start_lat'], edge['id_start_long']],
            [edge['id_end_lat'], edge['id_end_long']]
        ],
        color="blue",
        weight=2,
        tooltip=f"Total Sum: {edge['total_sum']}"
    ).add_to(m)

# Add bus stops as markers
for _, stop in busstop_data_df.iterrows():
    folium.CircleMarker(
        location=[stop['latitude'], stop['longitude']],
        radius=3,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.7,
        tooltip=f"Total: {stop['total']}"
    ).add_to(m)

# Save the map as an HTML file
output_html = 'C:/Users/pjy03/OneDrive/바탕 화면/university/KHUDA/심화프로젝트/혼잡정류장/busstop_updated.html'
m.save(output_html)

print(f"Updated CSV file saved at: {output_file}")
print(f"Map HTML file saved at: {output_html}")


In [ ]:

## 승하차 인원을 엣지 길이로 나눠서 밀도 계산

# Load the updated busstop file
busstop_file = 'C:/Users/pjy03/OneDrive/바탕 화면/university/KHUDA/심화프로젝트/혼잡정류장/busstop_edited.csv'
busstop_df = pd.read_csv(busstop_file)

# Ensure the required columns are present
required_columns = ['total_sum', 'distance']
if not all(col in busstop_df.columns for col in required_columns):
    raise ValueError(f"The file is missing required columns: {required_columns}")

# Calculate the busstop density
busstop_df['busstop_density'] = busstop_df['total_sum'] / busstop_df['distance']

# Save the updated file
output_file = 'C:/Users/pjy03/OneDrive/바탕 화면/university/KHUDA/심화프로젝트/혼잡정류장/busstop_real_final.csv'
busstop_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Updated file saved at: {output_file}")
